In [1]:
# DataGeneration.jl
include("GenerateFineCluster.jl")
include("GenerateCoarseClusterTree.jl")
include("GenerateCoarseCluster.jl")

using Main.GenerateFineCluster:read_fine_cluster, plot_fine_cluster, fine_cluster
using Main.GenerateCoarseClusterTree:read_coarse_cluster_tree, plot_coarse_cluster_tree, coarse_cluster_tree
using Main.GenerateCoarseCluster:coarse_cluster, plot_coarse_cluster1, plot_coarse_cluster2, read_coarse_cluster
using HDF5, ProgressBars, InvertedIndices
using MAT
using Statistics

# Function to read a dataset from HDF5 file
function read_datasets_and_concat(file, names)
    hfile = "/Users/ree/Documents/Research/RegimeSurrogate/code/ClusteringDynamical/data/" *file* ".hdf5"
    datasets = Dict()
    combined_data = []

    h5open(hfile, "r") do h5file
        for name in names
            dataset = read(h5file[name])
            datasets[name] = dataset
            combined_data = isempty(combined_data) ? dataset : vcat(combined_data, dataset)
        end
        dt = read(h5file["dt"])
        datasets["dt"] = dt
    end
        
    return combined_data, datasets["dt"][1], datasets
end

# Assuming 'data' has different features in rows
function standardize(data)
    # Subtract the mean from each column (feature)
    data_centered = data .- mean(data, dims=2)
    
    # Divide each column (feature) by its standard deviation
    data_standardized = data_centered ./ std(data, dims=2)
    
    return data_standardized
end

function read_data(file)
    hfile = h5open("/Users/ree/Documents/Research/RegimeSurrogate/code/ClusteringDynamical/data/" *file* ".hdf5")
    x = read(hfile["x"])
    dt = read(hfile["dt"])
    close(hfile)
    return x, dt
end

x = []
dt = []
files = ["potential_well_step_bigT"]
# files = ["enso_sigma2state"]
# feature_lists = [["T_C","T_E"]]
# for i in 1:length(files)
#     temp1, temp2, _ = read_datasets_and_concat(files[i],feature_lists[i])
#     push!(x, standardize(temp1))
#     push!(dt, temp2)
# end
for i in (1:length(files))
    temp1, temp2 = read_data(files[i])
    push!(x, temp1)
    push!(dt, temp2)
end

# # time lag
# files = ["potential_well_step_tlag2"]
# lagtime = 2
# laginter = 1
# sequence_length = div(lagtime, laginter) + 1  # length of time sequences

# # create sequence data
# X = x[1]
# X_new = zeros(size(X, 1), sequence_length, size(X, 2)- lagtime)
# for i in 1:(size(X, 2) - lagtime)
#     X_new[:,:,i] = X[:,i:laginter:i + lagtime]
# end
# X_new = reshape(X_new, :, size(X, 2) - lagtime)
# x[1] = X_new

In [2]:
# # ######## DANGER #########
# for i in 1:length(files)
#    fine_cluster(x[i]; n_clusters = 2000, file = files[i])
# end

X = []
Xc = []
for i in (1:length(files))
    temp1, temp2 = read_fine_cluster(files[i])
    push!(X, temp1)
    push!(Xc, temp2)
end

# Create a dictionary of the variables you want to save
vars_to_save = Dict(
    "X" => X[1],
    "Xc" => Xc[1]
)
# # matwrite("../data/enso_TcTe_finecluster.mat", vars_to_save)
# matwrite("../data/potential_well_step_finecluster_bigT.mat", vars_to_save)


# ######## DANGER #########
ntimescales = 8
interv_timescales = 1
qmin = 0.1
for i in 1:length(files)
    coarse_cluster_tree(X[i],dt[i],ntimescales,interv_timescales; file=files[i], q_min=qmin)
end

X_LN_array = []
adj_array = []
adj_mod_array = []
node_labels_array = []
edge_numbers_array = []
τ = []
for i in 1:length(files)
    temp1, temp2, temp3, temp4, temp5, temp6 = read_coarse_cluster_tree(files[i])
    push!(X_LN_array, temp1)
    push!(adj_array, temp2)
    push!(adj_mod_array, temp3)
    push!(node_labels_array, temp4)
    push!(edge_numbers_array, temp5)
    push!(τ, temp6)
end

# Create a dictionary of the variables you want to save
vars_to_save = Dict(
    "X_LN_array" => X_LN_array[1],
    "tau" => τ[1]
)

# Save the dictionary to a .mat file
# matwrite("../data/enso_TcTehWutauHov_Ludo_standard.mat", vars_to_save)
# matwrite("../data/enso_sigma2state_TcTe_Ludo_standard.mat", vars_to_save)
matwrite("../data/potential_well_step_Ludo_bigT_q01.mat", vars_to_save)

In [ ]:
# ######## DANGER #########
# qmins = [] # maximum numbers of clustering centers
# push!(qmins, [2 ,2, 2, 2, 2, 2, 2, 2])
# # push!(qmins, [3 ,3 ,3 ,4 ,4 ,4 ,4 , 4])
# # push!(qmins, [2 ,4 ,4 ,4 ,4 ,4 ,4 , 4])
# # push!(qmins, [2 ,2 ,2 ,4 ,4 ,4 ,4 , 2])  
# # push!(qmins, [3 ,3 ,3 ,3 ,3 ,3 ,3 ,3])
# indices = [1,2,3,4,5,6,7,8]
# for i in 1:length(files)
#     coarse_cluster(X[i], dt[i], indices, τ[i], qmins[i]; file=files[i])
# end

# X_LN_array2 = []
# Q_array = []
# Q_pert_array = []
# Pt_array = []
# Qt_array = []
# Qt_pert_array = []
# score = []
# for i in 1:4
#     temp1, temp2, temp3, temp4, temp5, temp6, temp7 = read_coarse_cluster(files[i])
#     push!(X_LN_array2, temp1)
#     push!(Q_array, temp2)
#     push!(Q_pert_array, temp3)
#     push!(Pt_array, temp4)
#     push!(Qt_array, temp5)
#     push!(Qt_pert_array, temp6)
#     push!(score, temp7)
# end

# # Create a dictionary of the variables you want to save
# vars_to_save = Dict(
#     "X_LN_array" => X_LN_array2[1],
#     "tau" => τ[1]
# )

# matwrite("../data/enso_sigma2state_TcTe_Ludo_standard_2cluster.mat", vars_to_save)


In [14]:
X[1]

120000-element Vector{Int64}:
   6
  12
  21
  34
  34
  15
  12
   2
   8
   6
  21
  15
   6
   ⋮
  40
  40
  24
  57
  46
  65
  65
  70
  65
 114
 133
 100

In [4]:
# Create a dictionary of the variables you want to save
vars_to_save = Dict(
    "X_LN_array" => X_LN_array2[1],
    "tau" => τ[1]
)

matwrite("../data/enso_sigma2state_TcTe_Ludo_standard_2cluseter.mat", vars_to_save)


In [6]:
τ

1-element Vector{Any}:
 [16.454312557279287, 7.616302681764542, 7.121551707698855, 4.924447055781102, 4.180547666871456, 3.1236280173652466, 3.0898097813964434, 2.7161872223435677]

In [9]:
X_LN_array2[1]

8-element Vector{Any}:
 [1 2 1; 1 2 1; … ; 2 1 2; 1 1 1]
 [2 2 2; 1 1 1; … ; 2 2 2; 1 1 1]
 [2 2 2; 1 1 2; … ; 2 2 2; 1 1 2]
 [2 2 2; 2 2 2; … ; 2 2 2; 1 1 1]
 [1 1 1; 1 1 1; … ; 1 1 1; 2 2 2]
 [2 2 2; 2 2 2; … ; 2 2 2; 1 1 1]
 [2 2 2; 2 2 2; … ; 2 2 2; 1 1 1]
 [1 1 1; 1 1 1; … ; 1 1 1; 2 2 2]

In [73]:
X_LN_array[1]

8-element Vector{Any}:
 [1, 1, 2, 2, 2, 2, 2, 2, 2, 2  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  4, 4, 4, 4, 3, 3, 3, 3, 3, 3]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3  …  6, 6, 6, 6, 5, 5, 5, 5, 5, 5]
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [6, 6, 6, 6, 6, 6, 6, 6, 6, 6  …  3, 3, 3, 3, 4, 4, 4, 4, 4, 4]
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4  …  5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [20]:
X_LN_array2[1]

120000×3 Matrix{Int64}:
 1  1  8
 3  3  7
 4  6  4
 6  1  1
 6  1  1
 4  3  7
 1  4  8
 1  5  2
 4  1  8
 6  4  3
 6  4  3
 4  6  4
 1  4  8
 ⋮     
 1  8  4
 1  8  4
 5  8  3
 3  8  8
 3  8  8
 1  2  2
 1  1  4
 1  2  2
 3  8  8
 4  7  2
 4  7  2
 1  2  2

In [6]:
# plot
using HDF5, GLMakie, GraphMakie, Statistics
using MarkovChainHammer.BayesianMatrix, LinearAlgebra

include("GenerateFineCluster.jl")
include("GenerateCoarseClusterTree.jl")
include("GenerateCoarseCluster.jl")

using Main.GenerateFineCluster:read_fine_cluster, plot_fine_cluster, plot_fine_cluster_array, fine_cluster
using Main.GenerateCoarseClusterTree:read_coarse_cluster_tree, plot_coarse_cluster_tree, coarse_cluster_tree
using Main.GenerateCoarseCluster:coarse_cluster, plot_coarse_cluster1, plot_coarse_cluster2, read_coarse_cluster
using HDF5, LaTeXStrings, GLMakie
using SparseArrays, NetworkLayout, Graphs, Printf, NetworkLayout, GraphMakie, Graphs
using Main.GenerateFineCluster:read_fine_cluster, plot_fine_cluster, fine_cluster
using Main.GenerateCoarseClusterTree:read_coarse_cluster_tree, plot_coarse_cluster_tree, coarse_cluster_tree
using Main.GenerateCoarseCluster:coarse_cluster, plot_coarse_cluster1, plot_coarse_cluster2, read_coarse_cluster
using HDF5, ProgressBars, InvertedIndices
using MultiDimensionalClustering.CommunityDetection, ProgressBars
##
function read_data(file)
    hfile = h5open(pwd()*"/data/" *file* ".hdf5")
    x = read(hfile["x"])
    dt = read(hfile["dt"])
    close(hfile)
    return x, dt
end

function read_datasets_and_concat(file, names)
    hfile = "/Users/ree/Documents/Research/RegimeSurrogate/code/ClusteringDynamical/data/" *file* ".hdf5"
    datasets = Dict()
    combined_data = []

    h5open(hfile, "r") do h5file
        for name in names
            dataset = read(h5file[name])
            datasets[name] = dataset
            combined_data = isempty(combined_data) ? dataset : vcat(combined_data, dataset)
        end
        dt = read(h5file["dt"])
        datasets["dt"] = dt
    end
        
    return combined_data, datasets["dt"][1], datasets
end

##
# seems incorrect. Not identifying the correct coarse cluster
file = "enso"
temp1, temp2, temp3, temp4, temp5, temp6 = read_coarse_cluster_tree(file)
X_LN_array1 = temp1
adj_array = temp2
adj_mod_array = temp3
node_labels_array = temp4
edge_numbers_array = temp5
τ = temp6

X, Xc = read_fine_cluster(file)
x, dt, _ = read_datasets_and_concat(file,["T_C","T_E"])

# Somehow incorrect
temp1, temp2, temp3, temp4, temp5, temp6, temp7 = read_coarse_cluster(file)
X_LN_array2 = temp1
Q_array = temp2
Q_pert_array = temp3
Pt_array = temp4
Qt_array = temp5
Qt_pert_array = temp6

# ##
# Q = mean(BayesianGenerator(X; dt=dt))
# Λ = eigvals(Q)
# X_LN_array1 = []
# Q_array = []
# PIs = []
# for i in ProgressBar(1:8)
#     q_min = 1e-16
#     τ = 1.0 / -real(Λ[end-i])
#     P = exp(Q*τ)  # the way to calculate P is changed here
#     F, G, H, PI = leicht_newman_with_tree(P, q_min)
#     X_LN = classes_timeseries(F, X)
#     Qtmp = mean(BayesianGenerator(X_LN; dt=dt))
#     push!(X_LN_array1, X_LN)
#     push!(Q_array, Qtmp)
#     push!(PIs, PI)
# end

##
# resolution=(5000, 4000)
n_timescales = length(τ)
t = 1

# ## 
# # Create Adjacency matrix
# adj_array = []
# adj_mod_array = []
# edge_numbers_array = []
# node_labels_array = []
# for t in 1:8
#     PI = PIs[t]
#     N = maximum([maximum([PI[i][1], PI[i][2]]) for i in eachindex(PI)])
#     adj = spzeros(Int64, N, N)
#     adj_mod = spzeros(Float64, N, N)
#     for i in ProgressBar(eachindex(PI))
#         ii = PI[i][1]
#         jj = PI[i][2]
#         modularity_value = PI[i][3]
#         adj[ii, jj] += 1
#         adj_mod[ii, jj] = modularity_value
#     end 
#     N = maximum([maximum([PI[i][1], PI[i][2]]) for i in eachindex(PI)])
#     nodel_labels = zeros(N)
#     for i in eachindex(PI)
#         nodel_labels[PI[i][1]] = PI[i][3]
#     end
#     push!(node_labels_array, nodel_labels)
#     push!(adj_array, adj)
#     push!(adj_mod_array, adj_mod)
#     push!(edge_numbers_array, length(PI))
# end

##
fig = Figure(resolution = (2850, 2102))
for (tt, t) ∈ enumerate([1, 4, 8])
    layout = Buchheim()
    colormap = :glasbey_hv_n256
    set_theme!(backgroundcolor=:white)

    ax11 = Axis(fig[1,tt]; title =  @sprintf("τ = %5.2f", τ[t]), titlesize = 40)
    G = SimpleDiGraph(adj_array[t])
    transparancy = 0.4 * adj_mod_array[t].nzval[:] / adj_mod_array[t].nzval[1] .+ 0.1
    nlabels_fontsize = 30
    edge_color = [(:red, transparancy[i]) for i in 1:edge_numbers_array[t]]
    nlabels = [@sprintf("%2.2f", node_labels_array[t][i]) for i in 1:nv(G)]
    graphplot!(ax11, G, layout=layout, nlabels=nlabels, node_size=80,
        node_color=(:orange, 0.9), edge_color=edge_color, edge_width=5,
        arrow_size=45, nlabels_align=(:center, :center),
        nlabels_textsize=nlabels_fontsize, tangents=((0, -1), (0, -1)))
    # cc = cameracontrols(ax11.scene)
    hidedecorations!(ax11)
    hidespines!(ax11);
    ax12 = LScene(fig[2,tt]; show_axis = false)
    res = 10
    markersize = 10.0
    scatter!(ax12, x[1, 1:res:end], x[2, 1:res:end], markersize=markersize, color=cgrad(colormap)[X_LN_array1[t][1:res:end]])
    # rotate_cam!(ax12.scene, (0.0, -10.5, 0.0))
    #=
    ax13 = LScene(fig[1,1]; show_axis = false)
    res = 10
    markersize = 10.0
    scatter!(ax13, x[1, 1:res:end], x[2, 1:res:end], x[3, 1:res:end], markersize=markersize, color=X[1:res:end], colormap = colormap)
    # rotate_cam!(ax13.scene, (0.0, -10.5, 0.0))
    =#

    ax = Axis(fig[3,tt])
    Q_graph = mean(BayesianGenerator(X_LN_array1[t], dt=dt))# Q_array[ii]
    g_Q = DiGraph(Q_graph')
    Q_prim = zeros(size(Q_graph))
    for i in 1:size(Q_graph)[1]
        Q_prim[:, i] .= -Q_graph[:, i] / Q_graph[i, i]
        Q_prim[i, i] = -1 / Q_graph[i, i]
    end
    elabels = string.([round(Q_prim[i]; digits=2) for i in 1:ne(g_Q)])
    # [@sprintf("%.0e", node_labels[i]) for i in 1:nv(G)]
    transparancy = [(Q_prim[i] > eps(10^6 * 1.0) ? maximum([Q_prim[i], 0.2]) : 0.0)  for i in 1:ne(g_Q)]
    elabels_color = [(:black, transparancy[i] > eps(100.0)) for i in 1:ne(g_Q)]
    #edge_color_Q = [(:black, transparancy[i]) for i in 1:ne(g_Q)]
    edge_color_Q = [(cgrad(colormap)[(i-1)÷(size(Q_graph)[1])+1], transparancy[i]) for i in 1:ne(g_Q)]
    node_color = [(cgrad(colormap)[i]) for i in 1:nv(g_Q)]
    edge_attr = (; linestyle=[:dot, :dash, :dash, :dash, :dot, :dash, :dash, :dash, :dot])
    elabels_fontsize = 40
    nlabels_fontsize = 40
    node_size = 80.0
    edge_width_Q = [10.0 for i in 1:ne(g_Q)]
    arrow_size_Q = [40.0 for i in 1:ne(g_Q)]
    node_labels_Q = repr.(1:nv(g_Q))
    if tt < 3
        kwargs_edges  = (;  elabels_color=elabels_color, elabels_textsize=elabels_fontsize, edge_color=edge_color_Q, edge_width=edge_width_Q, elabels=elabels)
    else 
        kwargs_edges  = (;  elabels_color=elabels_color, elabels_textsize=elabels_fontsize, edge_color=edge_color_Q, edge_width=edge_width_Q)
    end
    kwargs_nodes  = (; node_color=node_color, node_size=node_size, nlabels=node_labels_Q, nlabels_textsize=nlabels_fontsize, nlabels_align=(:center,:center))
    kwargs_arrows = (; arrow_size=arrow_size_Q)
    graphplot!(ax, g_Q; kwargs_edges..., kwargs_nodes..., kwargs_arrows..., elabels_distance=20, layout = Stress())
    hidedecorations!(ax)
    hidespines!(ax)
end
display(fig)
##
save("figure/curated_Figure2_enso.png", fig)
# ##
# τs = reverse((1.0 ./ -real.(Λ))[1:end-1])
# ts = []
# scores = []
# for i in [1, 4, 8]
#     @info "beginning"
#     τ = τs[i]
#     P = exp(Q*τ)
#     P⁺ = exp(Q*(τ+dt))
#     P⁻ = exp(Q*(τ-dt))
#     @info "matrix exponential"
#     q_min = 0.0
#     @info "L1"
#     F, _, _, _ = leicht_newman_with_tree(P, q_min)
#     @info "L2"
#     F⁺, _, _, _ = leicht_newman_with_tree(P⁺, q_min)
#     @info "L3"
#     F⁻, _, _, _ = leicht_newman_with_tree(P⁻, q_min)
#     @info "classes"
#     X_LN = classes_timeseries(F, X)
#     X_LN⁺ = classes_timeseries(F⁺, X)
#     X_LN⁻ = classes_timeseries(F⁻, X)
#     @info "label ordering"
#     score1, _ =  label_ordering(X_LN⁺, X_LN)
#     @info "label ordering 2"
#     score2, _ =  label_ordering(X_LN⁻, X_LN)
#     score = (score1 + score2) / 2
#     push!(scores, score)
#     push!(ts, τ)
# end

LoadError: SystemError: opening file "/Users/ree/Documents/Research/RegimeSurrogate/code/ClusteringDynamical/cluster/figure/curated_Figure2_enso.png": No such file or directory

In [37]:
# FiguresPaper.jl

include("GenerateFineCluster.jl")
include("GenerateCoarseClusterTree.jl")
include("GenerateCoarseCluster.jl")

using Main.GenerateFineCluster:read_fine_cluster, plot_fine_cluster, plot_fine_cluster_array, fine_cluster
using Main.GenerateCoarseClusterTree:read_coarse_cluster_tree, plot_coarse_cluster_tree, coarse_cluster_tree
using Main.GenerateCoarseCluster:coarse_cluster, plot_coarse_cluster1, plot_coarse_cluster2, read_coarse_cluster
using HDF5, LaTeXStrings, GLMakie
using SparseArrays, NetworkLayout, Graphs, Printf, NetworkLayout, GraphMakie, Graphs

##
function read_data(file)
    hfile = h5open("../data/" *file* ".hdf5")
    x = read(hfile["x"])
    dt = read(hfile["dt"])
    close(hfile)
    return x, dt
end

function plot_fine_clusterP(x,X,title,figure_number,mks,res,azimuth,elevation)
    fig = Figure(resolution=(5000, 3000))
    colormap = :glasbey_hv_n256
    for i in 1:3
        ax = Axis3(fig[1, 2*(i-1)+1:2*i], xticklabelsize=40, yticklabelsize=40, zticklabelsize=40, xlabelsize=60, ylabelsize=60, zlabelsize=60, azimuth = azimuth[i], elevation=elevation[i])
        scatter!(ax, x[i][1, 1:res[i]:end], x[i][2, 1:res[i]:end], x[i][3, 1:res[i]:end], color=cgrad(colormap)[(X[i][1:res[i]:end] .% 256 .+1)]; markersize=mks[i])
        Label(fig[1, 2*(i-1)+1:2*i, Top()], title[i]; textsize=100)
    end
    for i in 1:1
        ax = Axis3(fig[2, 3*(i-1)+1:3*i], xticklabelsize=40, yticklabelsize=40, zticklabelsize=40, xlabelsize=60, ylabelsize=60, zlabelsize=60, azimuth = azimuth[i+3], elevation=elevation[i+3])
        scatter!(ax, x[i+3][1, 1:res[i+3]:end], x[i+3][2, 1:res[i+3]:end], x[i+3][3, 1:res[i+3]:end], color=cgrad(colormap)[(X[i+3][1:res[i+3]:end] .% 256 .+1)]; markersize=mks[i+3])
        Label(fig[2, 3*(i-1)+1:3*i, Top()], title[i+3]; textsize=100)
    end
    colgap!(fig.layout, 0)
    rowgap!(fig.layout, 0)
    save("/Users/ree/Documents/Research/RegimeSurrogate/fig/" * string(figure_number[1])*string(figure_number[2]) * ".png", fig)
end

function plot_coarse_cluster1P(x, X, τ, dt, X_LN_array, Q_array, score, title, figure_number; res = 1, mks = 5, azimuth=0.0pi, elevation=0.0pi, Lt=1000)
    indices_len = length(τ)
    g_Qarr = []
    kwargs_edges = []
    kwargs_nodes = []
    kwargs_arrows = []
    colormap = :glasbey_hv_n256
    for ii = 1:indices_len
        Q_graph = Q_array[ii]
        g_Q = DiGraph(Q_graph')
        Q_prim = zeros(size(Q_graph))
        for i in 1:size(Q_graph)[1]
            Q_prim[:, i] .= -Q_graph[:, i] / Q_graph[i, i]
            Q_prim[i, i] = -1 / Q_graph[i, i]
        end
        elabels = string.([round(Q_prim[i]; digits=2) for i in 1:ne(g_Q)])
        # [@sprintf("%.0e", node_labels[i]) for i in 1:nv(G)]
        transparancy = [Q_prim[i] for i in 1:ne(g_Q)]
        elabels_color = [(:black, transparancy[i] > eps(100.0)) for i in 1:ne(g_Q)]
        #edge_color_Q = [(:black, transparancy[i]) for i in 1:ne(g_Q)]
        edge_color_Q = [(cgrad(colormap)[(i-1)÷(size(Q_graph)[1])+1], transparancy[i]) for i in 1:ne(g_Q)]
        node_color = [(cgrad(colormap)[i]) for i in 1:nv(g_Q)]
        edge_attr = (; linestyle=[:dot, :dash, :dash, :dash, :dot, :dash, :dash, :dash, :dot])
        elabels_fontsize = 40
        nlabels_fontsize = 40
        node_size = 200.0
        edge_width_Q = [10.0 for i in 1:ne(g_Q)]
        arrow_size_Q = [40.0 for i in 1:ne(g_Q)]
        node_labels_Q = repr.(1:nv(g_Q))
        push!(kwargs_edges, (; elabels=elabels, elabels_color=elabels_color, elabels_textsize=elabels_fontsize, edge_color=edge_color_Q, edge_width=edge_width_Q))
        push!(kwargs_nodes, (; node_color=node_color, node_size=node_size, nlabels=node_labels_Q, nlabels_textsize=nlabels_fontsize))
        push!(kwargs_arrows, (; arrow_size=arrow_size_Q))
        push!(g_Qarr, g_Q)
    end
    set_theme!(backgroundcolor=:white)
    fig = Figure(resolution=(3000, 2000))
    fct = 10
    for i = 1:indices_len
        ax = Axis(fig[2:fct+1, i]; title=latexstring("\$\\textbf{t=$(round(τ[i],sigdigits=3))}\\,\\textbf{(\\Delta=$(round(score[i],sigdigits=3)))}\$"), titlesize=60)
        hidedecorations!(ax); hidespines!(ax)
        graphplot!(ax, g_Qarr[i]; kwargs_edges[i]..., kwargs_nodes[i]..., kwargs_arrows[i]...)
    end
    for i = 1:indices_len
        ax = Axis3(fig[fct+2:2*fct+1, i], xticklabelsize=40, yticklabelsize=40, zticklabelsize=40, xlabelsize=40, ylabelsize=40, zlabelsize=40, azimuth = azimuth, elevation=elevation)

        #scatter!(ax, x[1, 1:res:end], x[2, 1:res:end], x[3, 1:res:end], color=cgrad(colormap)[X_LN_array[i][1:res:end,1]]; markersize=mks)
        scatter!(ax, x[1, 1:res:end], x[2, 1:res:end], x[3, 1:res:end], color=cgrad(colormap)[X_LN_array[i][1:res:end,2]]; markersize=mks)
        #scatter!(ax, x[1, 1:res:end], x[2, 1:res:end], x[3, 1:res:end], color=cgrad(colormap)[X_LN_array[i][1:res:end,3]]; markersize=mks)
    end
    for i = 1:indices_len
        ax = Axis(fig[2*fct+2:3*fct+1, i], xticklabelsize=40, yticklabelsize=40, xlabelsize=40, ylabelsize=40, ylabel=latexstring("\$i_c\$"), xlabel=L"t")
 
        t_ax = [dt:dt:Lt*dt...]
        lines!(ax,t_ax,X[1:Lt],color=cgrad(colormap)[X_LN_array[i][1:Lt,2]], linewidth=2)
    end
    Label(fig[1, 1:3, Top()], title; textsize=100)
    save("/Users/ree/Documents/Research/RegimeSurrogate/fig/" * string(figure_number[1])*string(figure_number[2]) * ".png", fig)
end
##

x = []
dt = []
files = ["naturerun_PWell"]
for i in 1:length(files)
    temp1, temp2 = read_data(files[i])
    push!(x, temp1)
    push!(dt, temp2)
end

X = []
Xc = []
for i in 1:length(files)
    temp1, temp2 = read_fine_cluster(files[i])
    push!(X, temp1)
    push!(Xc, temp2)
end

X_LN_array1 = []
adj_array = []
adj_mod_array = []
node_labels_array = []
edge_numbers_array = []
τ = []
for i in 1:length(files)
    temp1, temp2, temp3, temp4, temp5, temp6 = read_coarse_cluster_tree(files[i])
    push!(X_LN_array1, temp1)
    push!(adj_array, temp2)
    push!(adj_mod_array, temp3)
    push!(node_labels_array, temp4)
    push!(edge_numbers_array, temp5)
    push!(τ, temp6)
end

X_LN_array2 = []
Q_array = []
Q_pert_array = []
Pt_array = []
Qt_array = []
Qt_pert_array = []
score = []
for i in 1:length(files)
    temp1, temp2, temp3, temp4, temp5, temp6, temp7 = read_coarse_cluster(files[i])
    push!(X_LN_array2, temp1)
    push!(Q_array, temp2)
    push!(Q_pert_array, temp3)
    push!(Pt_array, temp4)
    push!(Qt_array, temp5)
    push!(Qt_pert_array, temp6)
    push!(score, temp7)
end

##
titles = [latexstring("\\textbf{\\textrm{Multi-well potential}}") 
        latexstring("\\textbf{\\textrm{Lorenz 63}}") 
        latexstring("\\textbf{\\textrm{Newton-Leipnik}}") 
        latexstring("\\textbf{\\textrm{Kuramoto-Sivashinsky}}") 
        latexstring("\\textbf{\\textrm{PIV}}")]

mks_vals = [5,5,3,3,20]
azimuth = [0.2pi, 0.05pi, 0.4pi, 1.2pi, 0.05pi]
elevation = [0.2pi, 0.2pi, 0.4pi, 0.1pi, 0.2pi]
Lt = [5000, 5000, 20000, 5000, 1000]
for f_index in 1:length(files)
    figure_number = [3 f_index]
    plot_coarse_cluster1P(x[f_index], X[f_index], τ[f_index][[1,4,8]], dt[f_index], X_LN_array2[f_index], Q_array[f_index], score[f_index], titles[f_index], figure_number; res = 1,mks = mks_vals[f_index], azimuth=azimuth[f_index], elevation=elevation[f_index], Lt=Lt[f_index])
end


In [12]:
hfile = h5open("../data/" * file * "_coarse_cluster.hdf5")
# t_steps = read(hfile["t_steps"])

🗂️ HDF5.File: (read-only) ../data/naturerun_PWell_coarse_cluster.hdf5
├─ 🔢 Pt_array_tms=1
├─ 🔢 Pt_array_tms=2
├─ 🔢 Pt_array_tms=3
├─ 🔢 Q_pert_tms=1
├─ 🔢 Q_pert_tms=2
├─ 🔢 Q_pert_tms=3
├─ 🔢 Q_tms=1
├─ 🔢 Q_tms=2
├─ 🔢 Q_tms=3
├─ 🔢 Qt_array_tms=1
├─ 🔢 Qt_array_tms=2
├─ 🔢 Qt_array_tms=3
├─ 🔢 Qt_pert_tms=1
├─ 🔢 Qt_pert_tms=2
├─ 🔢 Qt_pert_tms=3
├─ 🔢 X_LN_tms=1
├─ 🔢 X_LN_tms=2
├─ 🔢 X_LN_tms=3
├─ 🔢 score_tms=1
├─ 🔢 score_tms=2
├─ 🔢 score_tms=3
└─ 🔢 t_steps